In [2]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\Kevin\Downloads\openslide-win64-20221217\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [3]:
import numpy as np
from PIL import Image
import math
import os

In [4]:
nssrc = r"\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained"
imgname = 'OTS_14684_2.ndpi'
ns_ndpi_src = os.path.join(nssrc,imgname)
slide = openslide.open_slide(ns_ndpi_src)
target_level = 0
target_dim = slide.level_dimensions[target_level]
width, height = target_dim
# unpad since it's white background anyways
tiles_x_unpad = int((width % 1024) / 2)
tiles_y_unpad = int((height % 1024) / 2)
# read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
num_rows = int(num_rows)
num_cols = int(num_cols)
print(num_rows)
print(num_cols)

142
93


In [ ]:
# def reshape_split(image:np.ndarray,kernel_size:tuple):
#     img_height,img_width,channels=image.shape
#     tile_height,tile_width = kernel_size
#     tiled_array = image.reshape(img_height//tile_height,
#                                 tile_height,
#                                 img_width//tile_width,
#                                 tile_width,
#                                 channels)
#     tiled_array = tiled_array.swapaxes(1,2)
#     return tiled_array

In [7]:
# tile to 1024 x 1024
tile_dstsrc = r'\\shelter\Kyu\unstain2stain\tiles\not_registrated\Unstained\OTS_14684_2'
target_level = 0
for row_idx in range(num_rows): # 0~num_rows-1
    for col_idx in range(num_cols): # 0~num_cols-1
        xcoord= tiles_x_unpad + 1024 * row_idx
        ycoord= tiles_y_unpad + 1024 * col_idx
        nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
        nsimage = np.array(nsimage)
        imgidx = row_idx * num_cols + (col_idx + 1)
        imgidx = f"{imgidx:04d}"
        imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
        imgname = imgcoord + imgidx + ".png"
        Image.fromarray(nsimage).save(os.path.join(tile_dstsrc,imgname))

### Now run inference on these tiles and then stitch the resulting tiles using stitch notebook